In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
import tensorflow as tf
from pyfaidx import Fasta
from sklearn.dummy import DummyClassifier
from sklearn.utils import shuffle, compute_class_weight
from utils import mapping_dict, seq2kmer, one_hot, seqToWordVec, seq2kmerMatrix
random.seed(12)
np.random.seed(12)
from sklearn.metrics import accuracy_score, f1_score
import joblib

1048645it [00:09, 112827.81it/s]


In [2]:
# Extract exons only
gtf = pd.read_csv('/Users/callummacphillamy/PhD/Reference_Genomes/dog_10K/GCF_000002285.5_Dog10K_Boxer_Tasha_genomic.gtf',
                  comment='#',
                  header=None,
                  sep='\t',
                  index_col=None)
with open('/Users/callummacphillamy/PhD/Reference_Genomes/dog_10K/GCF_000002285.5_Dog10K_Boxer_Tasha_genomic.EXONS-ONLY.bed', 'w') as bed:
    for row in gtf.itertuples():
        if str(row[3]).lower() == 'exon':
            assert str(row[3]).lower() == 'exon'
            bed.write(f'{row[1]}\t{row[4]}\t{row[5]}\t{row[3]}\n')
bed.close()

## Subtract Exons from ChIP regions
```shell
bedtools subtract -a Bt_AllDatasets.consensus.MERGED.bed -b GCF_002263795.1_ARS-UCD1.2_genomic_EXONS_ONLY.bed > Bt_AllDatasets.consensus.MERGED.No-EXONS.bed
bedtools subtract -a chip_results/Ss_AllDatasets.consensus.MERGED.bed -b GCF_000003025.6_Sscrofa11.1_genomic_EXONS_ONLY.bed > chip_results/Ss_AllDatasets.consensus.MERGED.No-EXONS.bed
#bedtools subtract -a chip_results/Clf_H3K27ac_Villar_consensus.MERGED.bed -b GCF_014441545.1_ROS_Cfam_1.0_genomic_EXONS_ONLY.bed > Clf_H3K27ac_Villar_consensus.MERGED.No-EXONS.bed
bedtools subtract -a chip_results/Clf_H3K27ac_Villar_consensus_0.5_reciprocal_overlap.MERGED.bed -b GCF_000002285.5_Dog10K_Boxer_Tasha_genomic.EXONS-ONLY.bed > Clf_H3K27ac_Villar_consensus.MERGED.No-EXONS.bed
```

## Subtract ChIP regions from candidate regions
```shell
bedtools subtract -a predictions/200bp/ARS-UCD1.2.200bp.candidate.regions.chrALL.200bp.20bp.step.tsv -b Bt_AllDatasets.consensus.MERGED.No-EXONS.bed > predictions/200bp/ARS-UCD1.2.200bp.candidate.regions.chrALL.200bp.20bp.step.No-ChIP.No.EXONS.bed
bedtools subtract -a Sscrofa11.1.200bp.cand_regions.20bpStep.chrAll.200bp.20bpStep.bed -b chip_results/Ss_AllDatasets.consensus.MERGED.No-EXONS.bed > Sscrofa11.1.200bp.cand_regions.20bpStep.chrAll.200bp.20bpStep.NO-ChIP.No-EXONS.bed
#bedtools subtract -a ROS_Cfam.1.candidate.regions.chrALL.200bp.20bpStep.No-Ns.bed -b Clf_H3K27ac_Villar_consensus.MERGED.No-EXONS.bed > ROS_Cfam.1.candidate.regions.chrALL.200bp.20bpStep.No-ChIP.No-EXONS.bed
bedtools subtract -a GCF_000002285.5_Dog10K_Boxer_Tasha_genomic.NON.EXONS.RM.bed -b Clf_H3K27ac_Villar_consensus.MERGED.No-EXONS.bed > Dog10K.candidate.regions.No-ChIP.No-EXONS.bed
```

In [7]:
# Divvy ChIP regions and/or candidate regions bed file for validation
chip_regions = pd.read_csv('/Users/callummacphillamy/PhD/Reference_Genomes/dog_10K/Clf_H3K27ac_Villar_consensus.MERGED.No-EXONS.bed',
                           sep='\t',
                           header=None,
                           index_col=None)
genome = Fasta('/Users/callummacphillamy/PhD/Reference_Genomes/dog_10K/GCF_000002285.5_Dog10K_Boxer_Tasha_genomic.fna')
n_count = 0
with open('/Users/callummacphillamy/PhD/Reference_Genomes/dog_10K/Clf_H3K27ac_Villar_consensus.MERGED.200bp.20bpStep.No-EXONS.No-Ns.bed', 'w') as bed:
    for row in tqdm(chip_regions.itertuples()):
        if row[3] - row[2] < 200:
            continue
        else:
            len_region = row[3] - row[2]
            #print(len_region, row[2], row[3])
            for i in range(0, (len_region - 200 + 1), 20):
                chro = row[1]
                start = row[2] + i
                stop = row[2] + i + 200
                seq = genome[f'{chro}'][start:stop]
                seq = str(seq).upper()
                if 'N' in seq:
                    n_count += 1
                elif 'N' not in seq:
                    bed.write(f'{chro}\t{start}\t{stop}\n')
bed.close()
print(n_count)

59245it [00:16, 3527.83it/s]

11


In [8]:
# Create filtered candidate regions bed file for validation
candidate_regions = pd.read_csv('/Users/callummacphillamy/PhD/Reference_Genomes/dog_10K/Dog10K.candidate.regions.No-ChIP.No-EXONS.bed',
                                sep='\t',
                                header=None,
                                index_col=None)
n_count = 0
n_le_200 = 0
with open('/Users/callummacphillamy/PhD/Reference_Genomes/dog_10K/Dog10K.candidate.regions.200bp.20bpStep.No-ChIP.No.EXONS.No-Ns.bed', 'w') as bed:
    for row in tqdm(candidate_regions.itertuples()):
        if row[3] - row[2] < 200:
            continue
        else:
            len_region = row[3] - row[2]
            #print(len_region, row[2], row[3])
            for i in range(0, (len_region - 200 + 1), 20):
                chro = row[1]
                start = row[2] + i
                stop = row[2] + i + 200
                seq = genome[f'{chro}'][start:stop]
                seq = str(seq).upper()
                if 'N' in seq:
                    n_count += 1
                elif 'N' not in seq:
                    bed.write(f'{chro}\t{start}\t{stop}\n')
bed.close()
print(n_count)

3814452it [05:39, 11238.83it/s]

3952


In [9]:
### CREATE BALANCED VALIDATION SET ###

# Load in candidate regions minus the ChIP-regions
# Cattle
candidate_regions = pd.read_csv('/Users/callummacphillamy/PhD/Reference_Genomes/canFam/ROS_Cfam.1.candidate.regions.chrALL.200bp.20bpStep.No-ChIP.No-EXONS.bed',
                                header=None,
                                sep='\t',
                                index_col=None)
candidate_regions_idx = []
for row in candidate_regions.itertuples():
    if row[3] - row[2] == 200:
        candidate_regions_idx.append(row[0])
candidate_regions_idx = np.array(candidate_regions_idx)

# Load in ChIP regions
chip_regions = pd.read_csv('/Users/callummacphillamy/PhD/Reference_Genomes/canFam/Clf_H3K27ac_Villar_consensus.MERGED.200bp.20bpStep.No-EXONS.No-Ns.bed',
                           header=None,
                           index_col=None,
                           sep='\t')
chip_regions_idx = np.arange(chip_regions.shape[0])
genome = Fasta('../canFam/GCF_014441545.1_ROS_Cfam_1.0_genomic.fna')

In [10]:
### PERMUTATION CELL ###

# Change these
#model_h = tf.keras.models.load_model('/Users/callummacphillamy/PhD/Deeplearning_benchmark/models/trained_models/VISTA/DeepEnhancerPlus200/adam/200/hg19_VISTA_balanced_samples_DeepEnhancerPlus_200bp_0.0005_200bp.h5')
#model_b = tf.keras.models.load_model('/Users/callummacphillamy/PhD/Deeplearning_benchmark/models/trained_models/VISTA/DeepEnhancerPlus200/adam/200/hg19.mm9_VISTA_balanced_samples_DeepEnhancerPlus_200bp_0.0005_200bp.h5')
#model_h = DummyClassifier(strategy='uniform', random_state=12)
#model_b = DummyClassifier(strategy='uniform', random_state=12)
#model_h.fit(shuf_hg19_test_X, shuf_hg19_test_y)
#model_b.fit(shuf_train_X, shuf_train_y)
model_h = joblib.load('/Users/callummacphillamy/PhD/Deeplearning_benchmark/models/trained_models/VISTA/hg19.VISTA.svm.200bp.joblib')
model_b = joblib.load('/Users/callummacphillamy/PhD/Deeplearning_benchmark/models/trained_models/VISTA/hg19.mm9.VISTA.svm.200bp.joblib')


model_name = 'SVM'

model_b_txt = open(f'/Users/callummacphillamy/PhD/Reference_Genomes/canFam/canFam_results/balanced_{model_name}_b.Clf.accuracy.csv', 'w')
model_h_txt = open(f'/Users/callummacphillamy/PhD/Reference_Genomes/canFam/canFam_results/balanced_{model_name}_h.Clf.accuracy.csv', 'w')



### DON'T CHANGE ###
model_b_txt.write('iter,accuracy,f1_score,training_set,model,model_name\n')
model_h_txt.write('iter,accuracy,f1_score,training_set,model,model_name\n')
for i in tqdm(range(1000)):

    # Take 500 random samples from candidate regions and from chip_regions
    neg_test_idx = np.random.choice(candidate_regions_idx, size=500)
    neg_test_df = candidate_regions.iloc[neg_test_idx, :3]
    pos_test_idx = np.random.choice(chip_regions_idx, size=500)
    pos_test_df = chip_regions.iloc[pos_test_idx, :3]

    neg_X = []
    for index in neg_test_df.itertuples():
        chro = index[1]
        start = index[2]
        stop = index[3]
        seq = genome[f'{chro}'][start:stop]
        seq = str(seq).upper()
        assert 'N' not in seq
        assert len(seq) == 200, 'Length of the sequence isn\'t 200bp'
        #try:
        neg_X.append(seq2kmer(seq))
        #except ZeroDivisionError:
        #    print(f'This sequence caused a zero division error:\n{seq}')
    neg_X = np.array(neg_X)

    pos_X = []
    for row in pos_test_df.itertuples():
        chro = row[1]
        start = row[2]
        stop = row[3]
        seq = genome[f'{chro}'][start:stop]
        seq = str(seq).upper()
        assert 'N' not in seq
        #try:
        pos_X.append(seq2kmer(seq))
        #except ZeroDivisionError:
        #    print(f'This sequence caused a zero division error:\n{seq}')
    pos_X = np.array(pos_X)

    neg_y = np.zeros(neg_X.shape[0])
    pos_y = np.ones(pos_X.shape[0])

    test_X = np.vstack((pos_X, neg_X))
    test_y = np.hstack((pos_y, neg_y))
    shuf_test_X, shuf_test_y = shuffle(test_X, test_y, random_state=12)

    yhat_h = model_h.predict(shuf_test_X)
    yhat_b = model_b.predict(shuf_test_X)
    yhat_h_score = accuracy_score(shuf_test_y, np.round(yhat_h))
    yhat_b_score = accuracy_score(shuf_test_y, np.round(yhat_b))
    yhat_h_f1 = f1_score(shuf_test_y, np.round(yhat_h))
    yhat_b_f1 = f1_score(shuf_test_y, np.round(yhat_b))
    model_b_txt.write(f'{i},{yhat_b_score},{yhat_b_f1},both,{model_name}_b,{model_name}\n')
    model_h_txt.write(f'{i},{yhat_h_score},{yhat_h_f1},human,{model_name}_h,{model_name}\n')

model_b_txt.close()
model_h_txt.close()

100%|██████████| 1000/1000 [46:29<00:00,  2.79s/it]


In [21]:
0/1

0.0